In [ ]:
import random

class Server:
    def __init__(self, power_w, load_factor):
        self.power_w = power_w
        self.load_factor = load_factor

    def variable_load_factor(self):
        base = self.load_factor
        fluctuation = random.uniform(-0.1, 0.1)  # ±10% random fluctuation
        return max(0, min(1, base + fluctuation))

    def power_consumption(self):
        l = self.variable_load_factor()
        return self.power_w * (0.6 * l + 0.4 * l * l)

class NetworkGear:
    def __init__(self, power_w):
        # No load factor because power utilization at >10% load is ~= power utilization at 100% load.
        self.power_w = power_w

    def power_consumption(self):
        return self.power_w

class DataCenterEnergyModel:
    def __init__(self, servers, network_gear, cooling_power_kw, lighting_power_kw):
        self.servers = servers
        self.network_gear = network_gear
        self.cooling_power_kw = cooling_power_kw
        self.lighting_power_kw = lighting_power_kw

    def total_energy_consumption(self, hours):
        self.compute_kwh = (sum(server.power_consumption() for server in self.servers) / 1000) * hours
        network_kwh = (sum(gear.power_consumption() for gear in self.network_gear) / 1000) * hours
        cooling_kwh = self.cooling_power_kw * hours
        lighting_kwh = self.lighting_power_kw * hours
        self.total_energy = self.compute_kwh + network_kwh + cooling_kwh + lighting_kwh
        return self.total_energy

    def pue(self):
        self.total_energy_consumption(24)
        return self.total_energy / self.compute_kwh if self.compute_kwh > 0 else float('inf')

# Example usage
servers = (
    [Server(100, 0.5) for _ in range(1000)] +
    [Server(500, 0.7) for _ in range(100)] +
    [Server(1000, 0.9) for _ in range(20)]
)
network_gear = (
    [NetworkGear(80) for _ in range(20)] +
    [NetworkGear(80) for _ in range(5)]
)

print(f"Total number of servers: {len(servers)}")
cooling_power_kw = 20  # Cooling system power consumption in kW
lighting_power_kw = 1  # Lighting power consumption in kW

data_center = DataCenterEnergyModel(servers, network_gear, cooling_power_kw, lighting_power_kw)
hours = 24  # Number of hours to calculate energy consumption for
total_energy = data_center.total_energy_consumption(hours)
pue = data_center.pue()
print(f"Total energy consumption for {hours} hours: {total_energy:.2f} kWh")
print(f"PUE: {pue:.2f}")


Total number of servers: 1120
Total energy consumption for 24 hours: 2665.98 kWh
PUE: 1.2633462422185473
